<a href="https://colab.research.google.com/github/Sorcerer14/Webscraping_CollegeDuniya/blob/main/Webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
urls = []
urls.append("https://collegedunia.com/moradabad-colleges") #Creating URL to directed site

no_of_pagedowns = 1

#Extracting Data from Website
#BeautifulSoup to extract contents from the website.
#requests

#Scrolling through the page dynamically
#selenium for using the chrome webdriver
#time to wait for a while and scroll down the page
#Also Needs Chrome Driver to be explicitely installed to work with dynamicaly scrolling down pages ###

#Pandas to export the stored dataframe as a csv

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd

for u in urls:  #iterates throught the list urls
    save_name = u[25:]+'.csv'       # specifies the name pandas has to store it as csv or xlxs
    save_name = save_name.replace("/","-")      # replaces / with - as the file name in the case would be invalid
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    browser = webdriver.Chrome(chrome_options=options)  # Starting browser with specified options
    browser.get(u)                              # opens the browser with the url from the iterator
    time.sleep(1)                               # waits for the page to load
    elem = browser.find_element_by_tag_name("body")     # gets the body tag from the html under which content is present
    while no_of_pagedowns:                      # scrolls the pages to the numeber of times
        elem.send_keys(Keys.PAGE_DOWN)          # pressed the down key
        time.sleep(0.4)                         # waits for the page to load
        no_of_pagedowns-=1                      # reduce the number of pagedowns by 1
    #Initiate a Pandas DataFrame with the Specific Columns
    df = pd.DataFrame(columns=['S No','College Name', 'Details','Course','Fees','Eligibility-Criteria','weblink'])
    html = browser.page_source              # gets the page source once it's loaded
    main_page_content = BeautifulSoup(html)     # extracts the features using bs4
    browser.close()                         # closes the broswer ( important !!)
    Content = []                            #Initialize an empty list
    # Iterate throught the no of list of colleges
    for i in range(len(main_page_content.find_all("div", {"class": "jsx-765939686 text-white"}))):
        paragraphs = main_page_content.find_all("div", {"class": "cdcms_college_highlights"})[i]
        link=url= paragraphs.a['href']              #gets the url of the colleges *NOT USED IN TASK BUT HELPFULL*
        name = paragraphs.text                      #gets the name of the college
        Content.append(paragraphs.text)             # Appends the name to the content list
        link = url                                
        response = requests.get(link, timeout=10)   # get to fetch the details inside the url
        page_content = BeautifulSoup(response.content, "html.parser")  # parses the data using bs4
        textContent = []                            #empty list and it has lots of important data
        # this has the important data we now extract from
        for k in range(len(page_content.find_all("div",{"class": "jsx-765939686 text-white"}))):
            paragraphs_new = page_content.find_all("div",{"class": "cdcms_college_highlights"})[k].text
            textContent.append(paragraphs_new)
        
        #This is most important part of this project where we join/clean all the data that we got from various sources
        #The insights are :-
        # 1) All colleges are from India and a particular area
        # 2) Content used is from top-most present user
        # 3) The table used first splitted into three parts i.e. course,fee-structure and eligibility-criteria
        # NOTE : There were lots of garbage data that was cleaned in below process

        for l in range(len(textContent)):
            start = textContent[l].find('India')
            start_weblink = textContent[l].find('http')
            http_val = textContent[l].find('http')
            if(start>=0):
                details = textContent[l][0:start+5]
                course = textContent[l][4:start+0]
                fees = textContent[l][4:start+1]
                eligibility_criteria = textContent[l][4:start+2]
                weblink = textContent[l][start_weblink:]
                while(details.find("\n")>0):
                    loc = details.find("\n")
                    details = details.replace('\n',',')
        deatails=details.replace(',    ',',')

        textContent = []
        for o in range(len(page_content.find_all("script",{"type": "application/ld+json"}))):
            paragraphs = page_content.find_all("script",{"type": "application/ld+json"})[o].text
            textContent.append(paragraphs)
        
        #Prints the name of the colleges
        #Create a list to store the detais and finally dumps into there Pandas dataframe
        print(name)
        list=[]
        list.append(i+1)
        list.append(name)
        list.append(details)
        list.append(course)
        list.append(fees)
        list.append(eligibility_criteria)
        list.append(weblink)
        #it now stores data in dataframe
        df.loc[i+1]=list
        #pandas stores it to the specified csv file from here.
        df.to_csv(r'/content/web_data/'+save_name,index=None,encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: use options instead of chrome_options
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
